In [1]:
# class nltk.parse.corenlp.GenericCoreNLPParser(url='http://localhost:9000', encoding='utf8', tagtype=None)

import numpy as np
import sys
import os
import nltk
import pandas as pd
import json
import re
# import Ipy_importer
import codecs
import subprocess
import copy
# nltk.download('words')
# nltk.download('maxent_ne_chunker')
# nltk.download('treebank')
from nltk.parse.stanford import StanfordParser
from nltk.parse.stanford import *
from nltk.parse.stanford import StanfordDependencyParser
from itertools import chain
import sys
import os
import gensim
import sklearn
import nltk
import numba
from scipy import spatial
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
'''
This section got the new attribute lexcicon
'''
with open("D:/Sneaker/new_new_attribute_lexicon.json", "r") as f:
    attribute_lexicon = json.load(f)
    print(type(attribute_lexicon))
    attribute_lexicon['Permeability']

    
temp1 = []
for k, v in attribute_lexicon.items():
    v = sorted(set(v),key=v.index)
    temp2 = []
    for j in v:
        if j not in temp1: 
            temp2.append(j)
#     print(temp2)
        temp1.append(j)
    attribute_lexicon[k] = temp2
# temp = []
# for v in attribute_lexicon.values():
#     for i  in v:
#         temp.append(i)
# attribute_lexicon = temp
# this is the new attribute lexicon got from ramin

<class 'dict'>


In [3]:
def look_n(dic, hao,  n):
    i = 0
    if hao == 'fanwei':
        for k, v in dic.items():
            i += 1
            if i <= n:
                print(k,v)
    if hao == 'zhiding':
        for k, v in dic.items():
            i += 1
            if i == n:
                print(k,v)
    

In [4]:
def create_new_POS(file_name):
    import re
    import numpy as np
    import nltk
    with open(file_name, 'r', encoding = "UTF-8-sig") as f:
        clean_review = f.readlines()
        clean_review = "".join(clean_review)
        clean_review = clean_review.split('\n')
#             dan_reviews = []
            
            
            # create a dic that key is product name and values is those reviews
             
        review_dic = {}
        for review in clean_review:
            p_name = review.split('\\')[0]
            
            p_review = review.split('>')[-1]
            if p_name not in review_dic:
                review_dic[p_name] = [p_review]
            else:
                review_dic[p_name].append(p_review)
            
            # enumerate the values inside the values of this dic
        for k, v in review_dic.items():
            c_reviews = []
            for line in v:
                line = line.strip()
                line = line.strip('\">')
                line = re.sub("[\s+\.\!\/_,$%^*(+\"\')]+|[+()?【】“”！，。？、~@#￥%……&*（）]+", " ",line)
                line = line.replace(']', '')
                line = line.replace('[', '')
                line = line.rstrip(' ')
                c_reviews.append(line)
            review_dic[k] = c_reviews
                

        for k,v in review_dic.items():
            review_dic[k] = sorted(set(review_dic[k]),key=review_dic[k].index)
        
        pure_review = copy.deepcopy(review_dic)
                
        for k,v in review_dic.items():
            POS_tags = []
            r_tokens = []
#             pure_review = []
            for review in v:
                
                tokens = nltk.word_tokenize(review)
#                     r_tokens.append(tokens)
#                 for tokens in r_tokens:
                POS_tags.append(nltk.pos_tag(tokens))
            review_dic[k] = POS_tags
            
        for k,v in review_dic.items():
            review_dic[k] = list(enumerate(review_dic[k]))
            
        for k,v in pure_review.items():
            pure_review[k] = list(enumerate(pure_review[k]))
                
            # add product name onto each element inside a list
        bitch = {}
        for k, v in review_dic.items():
            temp = []
            for i in v:
                bitch[str(k) + str(i[0])] = str(i[1])
                i = str(k) + str(i[0]) + ":" + str(i[1])
                temp.append(i)
                
            review_dic[k] = temp
            
        whore  = {}
        for k, v in pure_review.items():
            temp = []
            for i in v:
                whore[str(k) + str(i[0])] = str(i[1])
                i = str(k) + str(i[0]) + ":" + str(i[1])
                temp.append(i)
                
            pure_review[k] = temp
        
        
        
    return review_dic, pure_review, bitch, whore

In [5]:
review_dic, pure_review, bitch, whore = create_new_POS("D:/Sneaker/clean_review.txt")

In [6]:
# look_n(whore, 'fanwei', 3)
# for v in whore.values():
#     print(type(v))
#     break

desperation = {}
for k, v in whore.items():
    desperation[k] = v.split(" ")

In [7]:
look_n(desperation, 'fanwei', 3)

Reebok_Classic_Leather_Gum_Casual_Shoes0 ['I', 'love', 'the', 'old', 'school', 'style', 'and', 'these', 'sneakers', 'are', 'so', 'comfortable']
Reebok_Classic_Leather_Gum_Casual_Shoes1 ['A', 'true', 'classic', 'for', 'the', 'fans', 'of', 'sneakers', 'and', 'culture', 'Very', 'nice', 'made', 'and', 'the', 'materials', 'are', 'pretty', 'good', 'too']
Reebok_Classic_Leather_Gum_Casual_Shoes2 ['They', 'were', 'sneakers', 'I', 'wore', 'when', 'I', 'was', 'a', 'kid', 'so', 'I', 'always', 'like', 'going', 'retro', 'when', 'sneakers', 'r', 'rereleased']


In [41]:
def dinf_seed(dic, num):
    # the num is the number of loops you want to inplement the loop
    seed = ['on']
    new_seed = []
    for k , v in dic.items():
        o = 0
        for word in v:
            if word in seed:
                tmp = []
                tmp.append(v[o -1])
                try:
                    tmp.append(v[o + 1])
                except:
                    pass
                new_seed.append(tmp)
            o += 1
    return new_seed
            

In [42]:
a = dinf_seed(desperation, _)

In [43]:
def get_key(value, dic):
    return [k for k, v in dic.items() if value in v]

In [44]:
def get_new(old_seed, old_lexicon):
    new_lexicon = []
    for i in old_seed:
        if i[-1] in list(chain(*list(attribute_lexicon.values()))):
            for v in old_lexicon.values():
                if i[0] in v and i[-1] in v:
                    index1 = v.index(i[0])
                    index2 = v.index(i[-1])
                    tmp = v[index1-1: index2+1]
                    if tmp != []:
                        new_lexicon.append(tmp)
    return new_lexicon
                    
#             print(i[-1])
#             print(get_key(i[-1], attribute_lexicon))
        

In [45]:
b = get_new(a, desperation)

In [47]:
newnew = []
for i in b:
    if len(i) < 4:
#         if 'of' in i:
#             i.remove('of')
        if i != []:
            newnew.append(i)
                

In [49]:
print(newnew[0:5])

[['amazing', 'shoes', 'at'], ['comfortable', 'shoes', 'at'], ['these', 'shoes', 'at'], ['review', 'shoes', 'at'], ['wearing', 'shoes', 'at']]
